# Limpieza de datos de la DANAE filtrado por giro económico de la actividad

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

filepath = '/content/denue_inegi_11_.csv' # Path al dataset

data = pd.read_csv(filepath, encoding='latin-1') # Lectura del CSV con codificacion específica

In [ ]:
# Limpieza de las columnas innecesarias

clean_data = data.drop(['CLEE', 'NOM_ESTAB', 'RAZ_SOCIAL', 'TIPO_VIAL', 'NOM_VIAL', 'TIPO_V_E_1',
                       'NOM_V_E_1', 'TIPO_V_E_2', 'NOM_V_E_2', 'TIPO_V_E_3', 'NOM_V_E_3', 'NUMERO_EXT',
                       'LETRA_EXT', 'EDIFICIO', 'EDIFICIO_E', 'NUMERO_INT', 'LETRA_INT',
                       'TIPO_ASENT', 'NOMB_ASENT', 'TIPOCENCOM', 'NOM_CENCOM', 'NUM_LOCAL',
                       'COD_POSTAL', 'CVE_ENT', 'ENTIDAD', 'CVE_MUN', 'CVE_LOC',
                       'AGEB', 'MANZANA', 'TELEFONO', 'CORREOELEC', 'WWW',
                       'TIPOUNIECO', 'FECHA_ALTA'], axis=1)


In [ ]:
# Carga de actividades Manufactureras 2024 del archivo CSV y conversion a SHP

# print(clean_data['PER_OCU'].unique())
# print(clean_data['NOMBRE_ACT'].unique())
# print(pd.factorize(clean_data['NOMBRE_ACT'])[0])

# Claves delimitadas a través de informacion con referencia en: https://www.inegi.org.mx/app/mapa/denue/default.aspx
data_manufactura24 = clean_data.drop(clean_data[(clean_data['CODIGO_ACT'] < 310000)].index, axis=0)
data_manufactura24 = clean_data.drop(clean_data[(clean_data['CODIGO_ACT'] > 339999)].index, axis=0)
data_manufactura24['PER_OCU_NUMERIC'] = pd.factorize(data_manufactura24['PER_OCU'])[0]
data_manufactura24['NOMBRE_ACT_NUMERIC'] = pd.factorize(data_manufactura24['NOMBRE_ACT'])[0]
data_manufactura24['y'] = data_manufactura24['LATITUD'].astype(float)
data_manufactura24['x'] = data_manufactura24['LONGITUD'].astype(float)

data_manufactura24 = data_manufactura24.drop(['LATITUD', 'LONGITUD'], axis=1)

# print(data_manufactura24)

# Pasar cvs a shp:

data_manufactura24.to_csv("data_manufactura24.csv")

# Lee el archivo CSV
data_to_shp = pd.read_csv("/content/data_manufactura24.csv")

# Asegúrate de que tienes columnas de latitud y longitud (ajusta los nombres de las columnas si es necesario)
# Por ejemplo, si tienes columnas 'lat' y 'lon' para latitud y longitud:
data_to_shp['geometry'] = data_to_shp.apply(lambda row: Point(row['x'], row['y']), axis=1)

# Convierte el DataFrame de pandas a GeoDataFrame de geopandas
geo_clean_data = gpd.GeoDataFrame(data_to_shp, geometry='geometry')

# Establece el sistema de coordenadas adecuado (CRS).
# El CRS depende de los datos, un CRS común es WGS84 (EPSG:4326).
geo_clean_data.set_crs(epsg=4326, inplace=True)

# Guarda el GeoDataFrame como un archivo shapefile (.shp)
geo_clean_data.to_file("data_manufactura24.shp", driver='ESRI Shapefile')

In [ ]:
# Carga de actividades Mineras 2024 del archivo CSV y conversion a SHP

# Claves delimitadas a través de informacion con referencia en: https://www.inegi.org.mx/app/mapa/denue/default.aspx
data_mineria24 = clean_data.drop(clean_data[(clean_data['CODIGO_ACT'] < 210000)].index, axis=0)
data_mineria24 = clean_data.drop(clean_data[(clean_data['CODIGO_ACT'] > 219999)].index, axis=0)
data_mineria24['PER_OCU_NUMERIC'] = pd.factorize(data_mineria24['PER_OCU'])[0]
data_mineria24['NOMBRE_ACT_NUMERIC'] = pd.factorize(data_mineria24['NOMBRE_ACT'])[0]
data_mineria24['y'] = data_mineria24['LATITUD'].astype(float)
data_mineria24['x'] = data_mineria24['LONGITUD'].astype(float)

data_mineria24 = data_mineria24.drop(['LATITUD', 'LONGITUD'], axis=1)


# Pasar cvs a shp:

data_mineria24.to_csv("data_mineria24.csv")

# Lee el archivo CSV
data_to_shp = pd.read_csv("/content/data_mineria24.csv")

# Asegúrate de que tienes columnas de latitud y longitud (ajusta los nombres de las columnas si es necesario)
# Por ejemplo, si tienes columnas 'lat' y 'lon' para latitud y longitud:
data_to_shp['geometry'] = data_to_shp.apply(lambda row: Point(row['x'], row['y']), axis=1)

# Convierte el DataFrame de pandas a GeoDataFrame de geopandas
geo_clean_data = gpd.GeoDataFrame(data_to_shp, geometry='geometry')

# Establece el sistema de coordenadas adecuado (CRS).
# El CRS depende de los datos, un CRS común es WGS84 (EPSG:4326).
geo_clean_data.set_crs(epsg=4326, inplace=True)

# Guarda el GeoDataFrame como un archivo shapefile (.shp)
geo_clean_data.to_file("data_mineria24.shp", driver='ESRI Shapefile')

In [ ]:
# Carga de archivos tipo CSV para estrés hídrico y unión de los datasets

#data_mineria24['PER_OCU_NUMERIC'].unique()
estres = pd.read_csv("/content/Estres_Hid_DataFrame.csv")

estres['ESTRES_HID'].max()

data1 = pd.read_csv("/content/EstresAnual.csv")
data2 = pd.read_csv("/content/MapaEstres.csv")

data1 = data1.rename(columns={'Región': 'ORG_CUENCA'})

data_full = pd.merge(data1, data2, on='ORG_CUENCA', how='inner')

print(data_full)
data_full.to_csv("data_estres.csv")
